# Assignment #4: Extracting syntactic groups using recurrent networks
Author: Pierre Nugues

In this assignment, you will create a system to extract syntactic groups from a text. You will apply it to the CoNLL 2000 dataset. You will train your models with PyTorch.

Be aware that in PyTorch, the data matrices, by default, have an unconventional ordering with recurrent networks. To have a batch ordering similar to what we saw during the course, you must use the `batch_first=True` argument. See here https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_sequence.html and https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html

Before you start the assignment, please run the prerequisites from the prerequistites notebook.

## Objectives

The objectives of this assignment are to:
* Write a program to detect partial syntactic structures called groups or chunks
* Understand the principles of supervised machine learning techniques applied to language processing
* Write a short report of 2 to 3 pages on the assignment

## Preliminaries

### Imports

This instruction may solve installation conflicts

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
import bs4
import requests
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

import numpy as np
from numpy import dot
from numpy.linalg import norm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader

import conlleval
import math 
import functools

### Seeds
Making things reproduceable

In [3]:
random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)

### Constants

In [4]:
EMBEDDING_DIM = 100
LSTM_HIDDEN_DIM = 128

### The datasets

You may need to adjust the paths to load the datasets from your machine.

In [5]:
train_file = 'corpus/train.txt'
test_file = 'corpus/test.txt'

## Reading the files

You will now convert the dataset in a Python data structure. Read the functions below to load the datasets. They store the corpus in a list of sentences. Each sentence is a list of rows, where each row is a dictionary.

In [6]:
def read_sentences(file):
    """
    Creates a list of sentences from the corpus
    Each sentence is a string
    :param file:
    :return:
    """
    f = open(file).read().strip()
    sentences = f.split('\n\n')
    return sentences

In [7]:
def split_rows(sentences, column_names):
    """
    Creates a list of sentence where each sentence is a list of lines
    Each line is a dictionary of columns
    :param sentences:
    :param column_names:
    :return:
    """
    new_sentences = []
    for sentence in sentences:
        rows = sentence.split('\n')
        sentence = [dict(zip(column_names, row.split())) for row in rows]
        new_sentences.append(sentence)
    return new_sentences

### Loading dictionaries

The CoNLL 2000 files have three columns: The wordform, `form`, its part of speech, `pos`, and the tag denoting the syntactic group also called the chunk tag, `chunk`.

In [8]:
column_names = ['form', 'pos', 'chunk']

We load the corpus as a list of dictionaries

In [9]:
train_sentences = read_sentences(train_file)
train_dict = split_rows(train_sentences, column_names)
train_dict[10:11]

[[{'form': 'He', 'pos': 'PRP', 'chunk': 'B-NP'},
  {'form': 'reckons', 'pos': 'VBZ', 'chunk': 'B-VP'},
  {'form': 'the', 'pos': 'DT', 'chunk': 'B-NP'},
  {'form': 'current', 'pos': 'JJ', 'chunk': 'I-NP'},
  {'form': 'account', 'pos': 'NN', 'chunk': 'I-NP'},
  {'form': 'deficit', 'pos': 'NN', 'chunk': 'I-NP'},
  {'form': 'will', 'pos': 'MD', 'chunk': 'B-VP'},
  {'form': 'narrow', 'pos': 'VB', 'chunk': 'I-VP'},
  {'form': 'to', 'pos': 'TO', 'chunk': 'B-PP'},
  {'form': 'only', 'pos': 'RB', 'chunk': 'B-NP'},
  {'form': '#', 'pos': '#', 'chunk': 'I-NP'},
  {'form': '1.8', 'pos': 'CD', 'chunk': 'I-NP'},
  {'form': 'billion', 'pos': 'CD', 'chunk': 'I-NP'},
  {'form': 'in', 'pos': 'IN', 'chunk': 'B-PP'},
  {'form': 'September', 'pos': 'NNP', 'chunk': 'B-NP'},
  {'form': '.', 'pos': '.', 'chunk': 'O'}]]

## Embeddings

### Reading the embeddings

In [10]:
embedding_file = 'corpus/glove.6B.100d.txt'

Apply the function below that reads GloVe embeddings and store them in a dictionary, where the keys will be the words and the values, the embedding vectors.

In [11]:
def read_embeddings(file):
    """
    Return the embeddings in the from of a dictionary
    :param file:
    :return:
    """
    embeddings = {}
    glove = open(file, encoding='utf8')
    for line in glove:
        values = line.strip().split()
        word = values[0]
        vector = np.array(values[1:], dtype='float32')
        embeddings[word] = vector
    glove.close()
    return embeddings

In [12]:
# We read the embeddings
embeddings_dict = read_embeddings(embedding_file)
embedded_words = sorted(list(embeddings_dict.keys()))

In [13]:
'# words in embedding dictionary: {}'.format(len(embedded_words))

'# words in embedding dictionary: 400000'

### Understanding the embeddings

In [14]:
embedded_words[100000:100010]

['chording',
 'chordoma',
 'chordophones',
 'chords',
 'chore',
 'chorea',
 'chorene',
 'choreograph',
 'choreographed',
 'choreographer']

In [15]:
embeddings_dict['chords'][:20]

array([-0.51973,  1.0395 ,  0.20924,  0.16285,  0.7209 ,  0.81524,
       -0.34641, -0.76654, -0.49576,  0.24634,  0.44094,  0.37701,
       -0.16396,  0.2775 ,  0.16563,  0.43869, -1.0887 ,  0.12663,
        0.66916,  0.3578 ], dtype=float32)

Using a cosine similarity, write a `closest(target_word, embeddings, count=10)` that computes the 10 closest words to the words _table_, _france_, and _sweden_.

In [16]:
def closest2(target_word, embeddings, count=10):
  c_s = []
  tar = embeddings[target_word]
  tar_len = np.sqrt(np.sum(tar**2))
  for word in embedded_words:
    vec = embeddings[word]
    dot = np.dot(vec,tar)
    vec_len = np.sqrt(np.sum(vec**2))
    c_s.append((word,dot/(vec_len*tar_len)))
  c_s = sorted(c_s, key=lambda x: -x[1])[:count]
  return [tup[0] for tup in c_s]

In [17]:
closest2('france', embeddings_dict)

['france',
 'belgium',
 'french',
 'britain',
 'spain',
 'paris',
 'germany',
 'italy',
 'europe',
 'netherlands']

In [18]:
closest2('sweden', embeddings_dict)

['sweden',
 'denmark',
 'norway',
 'finland',
 'netherlands',
 'austria',
 'switzerland',
 'germany',
 'swedish',
 'belgium']

## Extracting the ${X}$ and ${Y}$ Lists of Symbols from the Datasets

For each sentence, you will build an input sequence, $\mathbf{x}$, corresponding to the words and an output one, $\mathbf{y}$, corresponding to the chunk tags.

Write a `build_sequences(corpus_dict, key_x='form', key_y='chunk', tolower=True)` function that, for each sentence, returns the $\mathbf{x}$ and $\mathbf{y}$ lists of symbols consisting of words and chunk tags. Set the words in lower case if `tolower` is true.

For the 11th sentence of the training set, you should have:<br/>
`x = ['he',  'reckons',  'the',  'current',  'account',  'deficit',  'will',  'narrow',  'to',  'only',  '#',  '1.8',  'billion',  'in',  'september',  '.']`

`y = ['B-NP', 'B-VP', 'B-NP', 'I-NP', 'I-NP', 'I-NP', 'B-VP', 'I-VP', 'B-PP', 'B-NP', 'I-NP', 'I-NP', 'I-NP', 'B-PP', 'B-NP', 'O']`

In [19]:
def build_sequences(corpus_dict, key_x='form', key_y='pos', tolower=True):
    X = []
    Y = []
    for sentence in corpus_dict:
      sen_x = []
      sen_y = []
      for word_dic in sentence:
        word = word_dic[key_x]
        if tolower:
          word = word.lower()
        sen_x.append(word)
        sen_y.append(word_dic[key_y])
      X.append(sen_x)
      Y.append(sen_y)
    return X, Y

In [20]:
X_train_symbs, Y_train_symbs = build_sequences(train_dict, key_x='form', key_y='chunk')

In [21]:
print(X_train_symbs[10])

['he', 'reckons', 'the', 'current', 'account', 'deficit', 'will', 'narrow', 'to', 'only', '#', '1.8', 'billion', 'in', 'september', '.']


In [22]:
print(Y_train_symbs[10])

['B-NP', 'B-VP', 'B-NP', 'I-NP', 'I-NP', 'I-NP', 'B-VP', 'I-VP', 'B-PP', 'B-NP', 'I-NP', 'I-NP', 'I-NP', 'B-PP', 'B-NP', 'O']


## Vocabulary

Create a vocabulary of all the words observed in the training set as well as in GloVe. You should find 401,464 different words. You will proceed in two steps.

First extract the list of unique words `words` from the CoNLL training set and the list of chunk tags, `chunks`. You will sort them

In [23]:
words = sorted(list(set(sum(X_train_symbs,[]))))
chunks = sorted(list(set(sum(Y_train_symbs, []))))

In [24]:
print('# words seen in training corpus:', len(words))
print('# Chunks tags seen:', len(chunks))

# words seen in training corpus: 17258
# Chunks tags seen: 22


In [25]:
words[4000:4010]

['casinos',
 'caspita',
 'caspita-brand',
 'cassettes',
 'cast',
 'castigated',
 'castigating',
 'castillo',
 'casting',
 'castro-medellin']

In [26]:
chunks[:10]

['B-ADJP',
 'B-ADVP',
 'B-CONJP',
 'B-INTJ',
 'B-LST',
 'B-NP',
 'B-PP',
 'B-PRT',
 'B-SBAR',
 'B-UCP']

Then, merge the list of unique CoNLL words with the words in the embeddings file. You will sort this list

In [27]:
vocabulary_words = sorted(list(set(embedded_words+words)))

In [28]:
print('# words in the vocabulary: embeddings and corpus:', len(vocabulary_words))

# words in the vocabulary: embeddings and corpus: 401464


In [29]:
vocabulary_words[200000:200010]

['joy',
 'joya',
 'joyal',
 'joyandet',
 'joyas',
 'joyce',
 'joycean',
 'joycelyn',
 'joyces',
 'joydeep']

## Index

Create the indices `word2idx`, `chunk2idx` and inverted indices `idx2word`, `idx2chunk` for the words and the chunk tags: i.e. you will associate each word with a number. You will use index 0 for the padding symbol and 1 for unknown words. This means that your first word will start at index 2. For the chunks, you will start at index 1.

In [30]:
idx2word = {(i+2):vocabulary_words[i] for i in range(len(vocabulary_words))}
idx2chunk = {(i+1):chunks[i] for i in range(len(chunks))}
word2idx = {vocabulary_words[i]:(i+2) for i in range(len(vocabulary_words))}
chunk2idx = {chunks[i]:(i+1) for i in range(len(chunks))}

The word indices

In [31]:
print(list(word2idx.items())[:25])

[('!', 2), ('!!', 3), ('!!!', 4), ('!!!!', 5), ('!!!!!', 6), ('!?', 7), ('!?!', 8), ('"', 9), ('#', 10), ('##', 11), ('###', 12), ('#a', 13), ('#aabccc', 14), ('#b', 15), ('#c', 16), ('#cc', 17), ('#ccc', 18), ('#cccccc', 19), ('#ccccff', 20), ('#d', 21), ('#daa', 22), ('#dcdcdc', 23), ('#e', 24), ('#f', 25), ('#faf', 26)]


The chunk indices

In [32]:
print(chunk2idx)

{'B-ADJP': 1, 'B-ADVP': 2, 'B-CONJP': 3, 'B-INTJ': 4, 'B-LST': 5, 'B-NP': 6, 'B-PP': 7, 'B-PRT': 8, 'B-SBAR': 9, 'B-UCP': 10, 'B-VP': 11, 'I-ADJP': 12, 'I-ADVP': 13, 'I-CONJP': 14, 'I-INTJ': 15, 'I-NP': 16, 'I-PP': 17, 'I-PRT': 18, 'I-SBAR': 19, 'I-UCP': 20, 'I-VP': 21, 'O': 22}


## Embedding Matrix

Create a numpy matrix of dimensions $(M, N)$, where $M$ will be the size of the vocabulary: The unique words in the training set and the words in GloVe, and $N$, the dimension of the embeddings.
The padding symbol and the unknown word symbol will be part of the vocabulary at respectively index 0 and 1. 

Initialize the matrix with random values with the `np.random.uniform()`

In [33]:
# We add two dimensions for the padding symbol at index 0 and unknown words at index 1
embedding_matrix = np.random.uniform(-0.05, 0.05, (len(vocabulary_words) + 2, EMBEDDING_DIM))
# embedding_matrix = np.random.random((len(vocabulary_words) + 2, EMBEDDING_DIM))
# embedding_matrix = np.zeros((len(vocabulary_words) + 2, EMBEDDING_DIM))

The shape of your matrix is: (401466, 100).

In [34]:
embedding_matrix.shape

(401466, 100)

Fill the matrix with the GloVe embeddings when available. This means: Replace the random vector with an embedding when available. You will use the indices from the previous section. You will call `out_of_embeddings` the list of words in CoNLL, but not in the embedding list.

In [35]:
out_of_embeddings = []
temp = set(embedded_words)
for word in vocabulary_words:
  idx = word2idx[word]
  if word in temp:
    embedding_matrix[idx] = embeddings_dict[word]
  else:
    out_of_embeddings.append(word)

In [36]:
len(out_of_embeddings)

1464

In [37]:
out_of_embeddings[-10:]

["y'all",
 'yankus',
 'year-ago',
 'year-before',
 'year-earlier',
 'year-to-date',
 'yield-management',
 'zaishuo',
 'zarett',
 'zumbrunn']

Embeddings of the padding symbol, idx 0, random numbers

In [38]:
embedding_matrix[0][:10]

array([-0.03084805,  0.01221088, -0.00622723,  0.02853586,  0.02799758,
       -0.02274074, -0.02235357,  0.03018722,  0.04581394,  0.03759326])

Embeddings of the word _table_, the GloVe values

In [39]:
embedding_matrix[word2idx['table']][:10]

array([-0.61453998,  0.89692998,  0.56770998,  0.39102   , -0.22437   ,
        0.49035001,  0.10868   ,  0.27410999, -0.23833001, -0.52152997])

Embeddings of _zarett_, a word in CoNLL 2000, but not in GloVe, random numbers

In [40]:
embedding_matrix[word2idx['zarett']][:10]

array([-0.04485961, -0.01950363,  0.03356147, -0.02404349, -0.04000838,
        0.01959841, -0.03943566, -0.01355046,  0.00896135, -0.02441297])

## Creating the ${X}$ and ${Y}$ Sequences

You will now create the input and output sequences with numerical indices. First, convert the 
${X}_\text{train\_symbs}$ and ${Y}_\text{train\_symbs}$ 
lists of symbols in lists of numbers using the indices you created. Call them `X_train_idx` and `Y_train_idx`.

In [41]:
X_train_idx = []
Y_train_idx = []
for x, y in zip(X_train_symbs, Y_train_symbs):
  X_train_idx.append([word2idx[w] for w in x])
  Y_train_idx.append([chunk2idx[c] for c in y])

Word indices of the three first sentences

In [42]:
print(X_train_idx[:3])

[[107701, 189360, 358640, 291209, 193879, 388606, 143496, 362305, 353285, 56501, 328878, 126632, 187522, 364843, 148777, 152124, 326524, 454, 131007, 152124, 306232, 363097, 454, 144953, 362305, 331257, 43426, 347508, 189267, 155109, 200552, 55175, 63614, 154, 259236, 120001, 873], [97171, 269136, 358640, 143112, 262191, 219534, 154, 307829, 106548, 362305, 43426, 149626, 249511, 288933, 174855, 177388, 362305, 293204, 43426, 154301, 189360, 344283, 274536, 358640, 279589, 386150, 873], [88319, 54890, 304156, 372747, 349558, 152124, 344283, 174855, 72318, 139858, 88675, 358640, 97171, 154, 144970, 362305, 56361, 57639, 261034, 288933, 240241, 189360, 180283, 234487, 183252, 340448, 218722, 360423, 873]]


Chunk tag indices of the three first sentences

In [43]:
print(Y_train_idx[:3])

[[6, 7, 6, 16, 11, 21, 21, 21, 21, 6, 16, 16, 9, 6, 16, 7, 6, 22, 1, 7, 6, 6, 22, 11, 21, 21, 6, 16, 16, 7, 6, 16, 16, 6, 16, 16, 22], [22, 7, 6, 16, 6, 16, 6, 16, 16, 7, 6, 16, 16, 16, 11, 21, 21, 21, 6, 16, 7, 6, 7, 6, 16, 16, 22], [22, 6, 11, 6, 16, 7, 6, 11, 21, 21, 7, 6, 16, 6, 16, 11, 21, 6, 16, 16, 16, 7, 6, 16, 16, 16, 6, 16, 22]]


Now, pad the sentences using the `pad_sequences` function. After padding, the second sentence you look like (the indices are not necessarily the same).
```
x = [ 97171, 269136, 358640, 143112, 262191, 219534,    154, 307829, 106548,
        362305,  43426, 149626, 249511, 288933, 174855, 177388, 362305, 293204,
         43426, 154301, 189360, 344283, 274536, 358640, 279589, 386150,    873,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0]
y = [22,  7,  6, 16,  6, 16,  6, 16, 16,  7,  6, 16, 16, 16, 11, 21, 21, 21,
         6, 16,  7,  6,  7,  6, 16, 16, 22,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0]
```

You will call the results `X_train_padded` and `Y_train_padded`.

In [44]:
X_train_idx = list(map(torch.LongTensor, X_train_idx))
Y_train_idx = list(map(torch.LongTensor, Y_train_idx))

In [45]:
X_train_padded = pad_sequence(X_train_idx, batch_first=True)
Y_train_padded = pad_sequence(Y_train_idx, batch_first=True)

In [46]:
X_train_padded[1]

tensor([ 97171, 269136, 358640, 143112, 262191, 219534,    154, 307829, 106548,
        362305,  43426, 149626, 249511, 288933, 174855, 177388, 362305, 293204,
         43426, 154301, 189360, 344283, 274536, 358640, 279589, 386150,    873,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0])

In [47]:
Y_train_padded[1]

tensor([22,  7,  6, 16,  6, 16,  6, 16, 16,  7,  6, 16, 16, 16, 11, 21, 21, 21,
         6, 16,  7,  6,  7,  6, 16, 16, 22,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0])

## Network Architecture

Create your network consisting of one embedding layer, a simple recurrent neural network, either RNN or LSTM, and a linear layer. You will initialize the embedding layer with `embedding_matrix` using `from_pretrained()`. You may try other configurations after. As number of RNN/LSTM units use 128.

In [48]:
class Model(nn.Module):

    def __init__(self, embedding_matrix, embedding_dim, lstm_units, nbr_classes, bidi_lstm=False):
        super().__init__()
        self.embeddings = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix).float(), freeze=True, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, lstm_units, num_layers=1, batch_first=True, bidirectional=bidi_lstm)
        if not bidi_lstm:
            self.fc = nn.Linear(lstm_units, nbr_classes)
        else:
            self.fc = nn.Linear(2*lstm_units, nbr_classes)

    def forward(self, sentence):
        embeds = self.embeddings(sentence)
        lstm_out, _ = self.lstm(embeds)
        lstm_out = F.relu(lstm_out)
        logits = self.fc(lstm_out)
        return logits

Create your model

In [49]:
model1 = Model(embedding_matrix, EMBEDDING_DIM, LSTM_HIDDEN_DIM, len(chunks) + 1, bidi_lstm=True)

In [50]:
model1

Model(
  (embeddings): Embedding(401466, 100, padding_idx=0)
  (lstm): LSTM(100, 128, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=23, bias=True)
)

Write the loss `loss_fn` and optimizer `optimizer`.

In [51]:
loss_fn = nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.RMSprop(model1.parameters(), lr=0.001)

## Data Loaders

In [52]:
X_train = torch.LongTensor(X_train_padded)
Y_train = torch.LongTensor(Y_train_padded)

In [53]:
dataset = TensorDataset(X_train, Y_train)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

## A Few Experiments
Flattening the tensors

In [54]:
Y_train.size()

torch.Size([8936, 78])

In [55]:
Y_train.view(-1)

tensor([6, 7, 6,  ..., 0, 0, 0])

In [56]:
Y_train.view(-1).size()

torch.Size([697008])

In [57]:
Y_train_pred = model1(X_train)

In [58]:
Y_train_pred.size()

torch.Size([8936, 78, 23])

In [59]:
Y_train_pred.view(-1, Y_train_pred.size()[-1]).size()

torch.Size([697008, 23])

## Training the Model

We create a dictionary to store the accuracy and the loss. Th exact values are difficult to compute because of the padding symbols. We include the padding symbols in the computation. 

In [60]:
history = {}
history['accuracy'] = []
history['loss'] = []

We fit the model

In [ ]:
for epoch in range(10):
    train_loss = 0
    train_accuracy = 0
    word_cnt = 0
    batch_cnt = 0
    for X_batch, Y_batch in tqdm(dataloader):
        batch_cnt += 1
        Y_batch_pred = model1(X_batch)
        loss = loss_fn(Y_batch_pred.view(-1, Y_batch_pred.shape[-1]), Y_batch.view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_accuracy += torch.sum(torch.argmax(model1(X_train), dim=-1) == Y_train)
    history['accuracy'] += [train_accuracy/torch.numel(Y_train)]
    history['loss'] += [train_loss/batch_cnt]

And we visualize the training curves. Ideally, we would compare them with a validation set.

In [ ]:
acc = history['accuracy']
loss = history['loss']

epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.title('Training accuracies')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.title('Training losses')
plt.legend()

plt.show()

We try the model on a test sentence

In [ ]:
sentence = 'The United States might collapsez .'.lower().split()

Convert the sentence words to indices

In [ ]:
# Write your code
# The indexes or the unknown word idx
sentence_word_idxs = ...

The indices. Note the 1 at the end.

In [ ]:
print('Sentence', sentence)
print('Sentence word indexes', sentence_word_idxs)

Predict the chunks. Call the variable `sent_chunk_predictions`

In [ ]:
# Write your code
sent_chunk_predictions = ...

In [ ]:
sent_chunk_predictions.shape

The estimated probabilities of the first chunk

In [ ]:
F.softmax(sent_chunk_predictions[0], dim=-1)

In [ ]:
torch.argmax(F.softmax(sent_chunk_predictions, dim=-1), dim=-1)

We apply argmax to select the chunk

In [ ]:
for word_nbr, chunk_predictions in enumerate(sent_chunk_predictions):
    if int(sentence_word_idxs[word_nbr]) in idx2word:
        print(idx2word[int(sentence_word_idxs[word_nbr])], end=': ')
    else:
        print(sentence[word_nbr], '/ukn', end=': ')
    print(idx2chunk.get(int(torch.argmax(F.softmax(chunk_predictions, dim=-1), dim=-1))))

## Evaluating the Model

In [ ]:
test_sentences = read_sentences(test_file)
test_dict = split_rows(test_sentences, column_names)
test_dict[1:2]

We create the ${X}$ and ${Y}$ sequences of symbols

In [ ]:
X_test_symbs, Y_test_symbs = build_sequences(test_dict, key_x='form', key_y='chunk')
print('X_test:', X_test_symbs[1])
print('Y_test', Y_test_symbs[1])

Convert the ${X}$ symbol sequence into an index sequence and pad it. Call the results `X_test_idx` and `X_test_padded`.

In [ ]:
# Write your code
X_test_idx = []
for x in X_test_symbs:
    ...

In [ ]:
X_test_idx = map(torch.LongTensor, X_test_idx)

In [ ]:
X_test_padded = pad_sequence(X_test_idx, batch_first=True)

In [ ]:
print('X_test_padded:', X_test_padded[1])

In [ ]:
X_test_padded.shape

Predict the chunks. Call the result `Y_test_hat_probs`

In [ ]:
# Write your code
Y_test_hat_probs = ...

In [ ]:
print('Predictions', Y_test_hat_probs[1])

In [ ]:
Y_test_hat_probs = F.softmax(Y_test_hat_probs, dim=-1)

We now predict the whole test set and we store the results in each dictionary with the key `pchunk`

In [ ]:
for sent, y_hat_probs in zip(test_dict, Y_test_hat_probs):
    sent_len = len(sent)
    y_hat_probs = y_hat_probs[:sent_len]
    # y_hat = torch.argmax(y_hat_probs, dim=-1) # This statement sometimes predicts 0 (the padding symbol)
    y_hat = torch.argmax(y_hat_probs[:, 1:], dim=-1) + 1 # Never predicts 0
    for word, ner_hat in zip(sent, y_hat):
        word['pchunk'] = idx2chunk.get(int(ner_hat)) 
        if word['pchunk'] == None:
            print(sent)

A sentence example: `chunk` is the hand annotation and `pchunk` is the prediction.

In [ ]:
test_dict[1]

We save the test set in a file to evaluate the performance of our model.

In [ ]:
column_names = ['form', 'pos', 'chunk', 'pchunk']

In [ ]:
def save(file, corpus_dict, column_names):
    """
    Saves the corpus in a file
    :param file:
    :param corpus_dict:
    :param column_names:
    :return:
    """
    i = 0
    with open(file, 'w', encoding='utf8') as f_out:
        i += 1
        for sentence in corpus_dict:
            sentence_lst = []
            for row in sentence:
                items = map(lambda x: row.get(x, '_'), column_names)
                sentence_lst += ' '.join(items) + '\n'
            sentence_lst += '\n'
            f_out.write(''.join(sentence_lst))

In [ ]:
outfile = 'test_model1.out'

In [ ]:
save(outfile, test_dict, column_names)

In [ ]:
lines = open(outfile, encoding='utf8').read().splitlines()
res = conlleval.evaluate(lines)
chunker_score = res['overall']['chunks']['evals']['f1']
chunker_score

In [ ]:
# Results may slightly vary depending on the run
# 0.8650974227443842 lstm nontrainable 15 epochs
# 0.8579701751845953 lstm trainable 15 epochs
# 0.9015216169521867 lstm bidi nontrainable 15 epochs
# 0.9000310655483068 lstm bidi trainable 15 epochs

## Experiments

You will carry out experiments with two different recurrent networks: RNN and LSTM. You will also try at least one set of parameters per network, i.e. two experiments, one with a RNN and one with a LSTM. To run a RNN, just replace the LSTM class with RNN. As baseline, a simple solution you consider a starting point, please report the baseline figures from CoNLL 2000: https://aclanthology.org/W00-0726.pdf. 

In your report, you will present your results in a table like this one:

|Method|Parameters|Score|
|------|-----|-----|
|Baseline|  xx | xx |
|RNN|  xx |xx |
|RNN |  xx |xx |
|LSTM |  xx |xx |
|LSTM |  xx |xx |
|  Akbik et al.|  xx|xx |

## Turning in your assignment

Now your are done with the program. To complete this assignment, you will:
1. Write a short individual report on your program. You will describe the architecture your used the different experiments you carried out and your results.
2. Read the article, <a href="https://www.aclweb.org/anthology/C18-1139"><i>Contextual String Embeddings for Sequence Labeling</i></a> by Akbik et al. (2018) and outline the main differences between their system and yours. A LSTM is a type of recurrent neural network, while CRF is a sort of beam search. You will tell the performance they reach on the corpus you used in this laboratory.

Submit your report as well as your notebook (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, you can either
1. Write directly your text in Canvas, or
2. Use Latex and Overleaf (www.overleaf.com). This will probably help you structure your text. You will then upload a PDF file in Canvas.

The submission deadline is October 13, 2023.